In [1]:
# Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from plotly.offline import iplot

# Greykite functions
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [10]:
nyc_df = pd.read_csv('nyc_data.csv')
future_df = pd.read_csv('future.csv')
df = pd.concat([nyc_df, future_df])
df = df.reset_index(drop = True)

df = df.rename(columns = {'Demand': 'y'})
df

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2015,720.000885,0,0,0,3.68,41.305
1,1/2/2015,581.276773,0,0,0,4.73,131.574
2,1/3/2015,754.117039,0,0,0,7.23,162.700
3,1/4/2015,622.252774,0,0,0,10.96,160.281
4,1/5/2015,785.373319,0,0,0,6.92,51.077
...,...,...,...,...,...,...,...
2218,1/27/2021,NaN,0,0,0,3.33,39.664
2219,1/28/2021,NaN,0,0,0,1.67,195.314
2220,1/29/2021,NaN,0,0,0,-2.78,235.894
2221,1/30/2021,NaN,0,0,0,1.11,152.752


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          2223 non-null   object 
 1   y             2192 non-null   float64
 2   Easter        2223 non-null   int64  
 3   Thanksgiving  2223 non-null   int64  
 4   Christmas     2223 non-null   int64  
 5   Temperature   2223 non-null   float64
 6   Marketing     2223 non-null   float64
dtypes: float64(3), int64(3), object(1)
memory usage: 121.7+ KB


## Silverkite Params

In [13]:
metadata = MetadataParam(time_col = 'Date',
                          value_col = 'y',
                          freq = 'D',
                          train_end_date = pd.to_datetime('2020-12-31'))
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [19]:
growth = {'growth_term': ['linear', 'quadratic', 'sqrt']}
growth

{'growth_term': ['linear', 'quadratic', 'sqrt']}

In [20]:
seasonality = {'yearly_seasonality': 'auto',
               'quarterly_seasonality': 'auto',
               'monthly_seasonality': 'auto',
               'weekly_seasonality': 'auto',
               'daily_seasonality': 'auto'}
seasonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [24]:
get_available_holiday_lookup_countries(['US'])

get_available_holidays_across_countries(countries = ['US'],
                                        year_start=2015,
                                        year_end=2021)

['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [64]:
events = {'holidays_to_model_separately': ["New Year's Day"],
          'holiday_lookup_countries': ['US'],
          'holiday_pre_num_days': 2,
          'holiday_post_num_days': 2,
          'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)},
          'daily_event_df_dict': {
              'elections': pd.DataFrame({
                  'date': ['2016-11-08', '2020-11-30'],
                  'event_name': 'elections'
              })
          }}

events

{'holidays_to_model_separately': ["New Year's Day"],
 'holiday_lookup_countries': ['US'],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)},
 'daily_event_df_dict': {'elections':          date event_name
  0  2016-11-08  elections
  1  2020-11-30  elections}}

In [65]:
changepoints = {'changepoints_dict': {'method': 'auto'}}
changepoints

{'changepoints_dict': {'method': 'auto'}}

In [66]:
regressors = {'regressor_cols': ['Easter', 'Temperature', 'Marketing']}
regressors

{'regressor_cols': ['Easter', 'Temperature', 'Marketing']}

In [67]:
lagged_regressors = {'lagged_regressor_dict': {'Temperature': 'auto',
                                               'Easter': 'auto',
                                               'Marketing': 'auto'}}
lagged_regressors

{'lagged_regressor_dict': {'Temperature': 'auto',
  'Easter': 'auto',
  'Marketing': 'auto'}}

In [68]:
autoregression = {'autoreg_dict': 'auto'}
autoregression

{'autoreg_dict': 'auto'}

In [69]:
custom = {'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
                                 {'fit_algorithm': 'ridge'},
                                 {'fit_algorithm': 'gradient_boosting'}]
        }
custom                         

{'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
  {'fit_algorithm': 'ridge'},
  {'fit_algorithm': 'gradient_boosting'}]}

## Model Building

In [70]:
model = ModelComponentsParam(growth=growth,
                             seasonality=seasonality,
                             events=events,
                             changepoints=changepoints,
                             regressors=regressors,
                             lagged_regressors=lagged_regressors,
                             autoregression=autoregression,
                             custom=custom)

In [71]:
evaluation_metric = EvaluationMetricParam(cv_selection_metric=
                    EvaluationMetricEnum.RootMeanSquaredError.name)

In [72]:
evaluation_period = EvaluationPeriodParam(
    cv_min_train_periods= df.shape[0]-211,
    cv_expanding_window=True,
    cv_max_splits=50,
    cv_periods_between_splits=16)

In [73]:
config = ForecastConfig(
    model_template = ModelTemplateEnum.SILVERKITE.name,
    forecast_horizon=31,
    metadata_param=metadata,
    model_components_param=model,
    evaluation_metric_param=evaluation_metric,
    evaluation_period_param=evaluation_period)

In [74]:
forecaster = Forecaster()
result = forecaster.run_forecast_config(df = df,
                                        config = config)

Fitting 8 folds for each of 9 candidates, totalling 72 fits


/home/salgado/scripts/time_series/.venv/lib64/python3.9/site-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3328: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

/home/salgado/scripts/time_series/.venv/lib64/python3.9/site-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3328: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

/home/salgado/scripts/time_series/.venv/lib64/python3.9/site-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3328: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin

In [76]:
cv_results = summarize_grid_search_results(
    grid_search=result.grid_search,
    decimals=2,
    score_func = EvaluationMetricEnum.RootMeanSquaredError.name)

cv_results['params'] = cv_results['params'].astype(str)
cv_results.set_index('params', drop=True, inplace=True)
cv_results

,rank_test_CORR,rank_test_R2,rank_test_MSE,rank_test_RMSE,rank_test_MAE,rank_test_MedAE,rank_test_MAPE,rank_test_MedAPE,rank_test_sMAPE,rank_test_Q80,...,std_test_OutsideTolerance5p,split0_train_OutsideTolerance5p,split1_train_OutsideTolerance5p,split2_train_OutsideTolerance5p,split3_train_OutsideTolerance5p,split4_train_OutsideTolerance5p,split5_train_OutsideTolerance5p,split6_train_OutsideTolerance5p,split7_train_OutsideTolerance5p,std_train_OutsideTolerance5p
params,,,,,,,,,,,,,,,,,,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",4,1,1,1,1,2,1,2,1,5,...,0.06,0.45,0.45,0.43,0.43,0.43,0.44,0.44,0.43,0.01
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,3,2,2,2,1,3,1,2,4,...,0.07,0.45,0.45,0.43,0.43,0.42,0.44,0.43,0.43,0.01
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,2,3,3,3,3,2,3,3,6,...,0.06,0.45,0.45,0.43,0.44,0.43,0.44,0.43,0.43,0.01
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,6,6,6,6,6,5,6,6,9,...,0.04,0.45,0.45,0.44,0.44,0.44,0.44,0.44,0.44,0.01
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",1,4,4,4,4,4,4,4,4,7,...,0.06,0.45,0.45,0.43,0.43,0.44,0.43,0.43,0.44,0.01
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,5,5,5,5,5,6,5,5,8,...,0.04,0.45,0.45,0.44,0.43,0.44,0.44,0.43,0.44,0.01
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",9,7,7,8,7,7,7,7,7,2,...,0.11,0.37,0.36,0.36,0.36,0.36,0.37,0.37,0.36,0.00
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",7,9,9,9,9,9,9,9,9,3,...,0.11,0.37,0.36,0.36,0.36,0.37,0.36,0.37,0.36,0.00
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",8,8,8,7,8,8,8,8,8,1,...,0.11,0.37,0.36,0.36,0.36,0.35,0.36,0.37,0.36,0.01


In [77]:
cv_results[["rank_test_RMSE", "mean_test_RMSE",
            "param_estimator__fit_algorithm_dict",
            "param_estimator__growth_term"]]

,rank_test_RMSE,mean_test_RMSE,param_estimator__fit_algorithm_dict,param_estimator__growth_term
params,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",1,47.30,{'fit_algorithm': 'linear'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,47.42,{'fit_algorithm': 'linear'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,47.43,{'fit_algorithm': 'linear'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,48.40,{'fit_algorithm': 'ridge'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",4,47.93,{'fit_algorithm': 'ridge'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,48.39,{'fit_algorithm': 'ridge'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",8,56.54,{'fit_algorithm': 'gradient_boosting'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",9,56.62,{'fit_algorithm': 'gradient_boosting'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'gradient_boosting'})]",7,56.53,{'fit_algorithm': 'gradient_boosting'},sqrt


In [83]:
result.backtest.plot()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': 'rgba(250, 43, 20, 0.7)'},
              'marker': {'color': 'rgba(250, 43, 20, 0.7)', 'size': 2.0},
              'mode': 'lines+markers',
              'name': 'Actual',
              'opacity': 1.0,
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0),
                          datetime.datetime(2015, 1, 3, 0, 0), ...,
                          datetime.datetime(2020, 12, 29, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2020, 12, 31, 0, 0)], dtype=object),
              'y': array([720.0008852, 581.2767734, 754.117039 , ..., 847.1233994, 857.5210429,
                          783.8914943])},
             {'line': {'color': 'rgba(0, 90, 181, 0.7)', 'dash': 'solid'},
              'name': 'Forecast',
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0),
                          datetime.datetime(2015, 1, 3, 0, 0), ...,
                          datetime.datetime(2020, 12, 29, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2020, 12, 31, 0, 0)], dtype=object),
              'y': array([715.10922045, 633.82160375, 728.68335721, ..., 719.20655646,
                          928.4446454 , 802.53128553])}],
    'layout': {'annotations': [{'arrowhead': 0,
                                'ax': -60,
                                'ay': 0,
                                'showarrow': True,
                                'text': 'Train End Date',
                                'x': Timestamp('2020-11-30 00:00:00'),
                                'xref': 'x',
                                'y': 0.97,
                                'yref': 'paper'}],
               'legend': {'traceorder': 'reversed'},
               'shapes': [{'line': {'color': 'rgba(100, 100, 100, 0.9)', 'width': 1.0},
                           'type': 'line',
                           'x0': Timestamp('2020-11-30 00:00:00'),
                           'x1': Timestamp('2020-11-30 00:00:00'),
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'paper'}],
               'showlegend': True,
               'template': '...',
               'title': {'text': 'Forecast vs Actual', 'x': 0.5},
               'xaxis': {'title': {'text': 'ts'}},
               'yaxis': {'title': {'text': 'y'}}}
})

In [88]:
from greykite.common.constants import SEASONALITY_REGEX
result.forecast.plot_components()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'lines',
              'name': 'y',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0),
                          datetime.datetime(2015, 1, 3, 0, 0), ...,
                          datetime.datetime(2020, 12, 29, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2020, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([720.0008852, 581.2767734, 754.117039 , ..., 847.1233994, 857.5210429,
                          783.8914943]),
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': 'trend',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0),
                          datetime.datetime(2015, 1, 3, 0, 0), ...,
                          datetime.datetime(2020, 12, 29, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2020, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x2',
              'y': array([ 0.00000000e+00,  5.98629951e-02,  1.17854971e-01, ..., -1.10115418e+02,
                           1.12349259e+02,  1.13546485e+02]),
              'yaxis': 'y2'},
             {'mode': 'lines',
              'name': 'WEEKLY_SEASONALITY',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([0., 1., 2., 3., 4., 5., 6.]),
              'xaxis': 'x3',
              'y': array([547.14203174, 486.22801888, 468.11146467, 429.85421599, 393.58030575,
                          384.39204041, 348.24292664]),
              'yaxis': 'y3'},
             {'mode': 'lines',
              'name': 'QUARTERLY_SEASONALITY',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([0.        , 0.01086957, 0.01098901, ..., 0.98888889, 0.98901099,
                          0.98913043]),
              'xaxis': 'x4',
              'y': array([4.76863754, 4.90257608, 4.90315391, ..., 4.4786571 , 4.48259201,
                          4.48642647]),
              'yaxis': 'y4'},
             {'mode': 'lines',
              'name': 'YEARLY_SEASONALITY',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([0.        , 0.00273973, 0.00547945, ..., 0.99178082, 0.99452055,
                          0.99726027]),
              'xaxis': 'x5',
              'y': array([213.15523253, 211.62445921, 210.01442642, ..., 216.85736743,
                          215.80037264, 214.5590707 ]),
              'yaxis': 'y5'},
             {'mode': 'lines',
              'name': 'events',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
                          datetime.datetime(2015, 1, 2, 0, 0),
                          datetime.datetime(2015, 1, 3, 0, 0), ...,
                          datetime.datetime(2020, 12, 29, 0, 0),
                          datetime.datetime(2020, 12, 30, 0, 0),
                          datetime.datetime(2020, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x6',
              'y': array([-15.57439242,  -1.40024442,   0.        , ..., -40.5247659 ,
                           -6.45972657, -47.51749303]),
              'yaxis': 'y6'},
             {'mode': 'lines',
              'name': 'residual',
              'opacity': 0.8,
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2015, 1, 1, 0, 0),
              

In [90]:
df

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2015,720.000885,0,0,0,3.68,41.305
1,1/2/2015,581.276773,0,0,0,4.73,131.574
2,1/3/2015,754.117039,0,0,0,7.23,162.700
3,1/4/2015,622.252774,0,0,0,10.96,160.281
4,1/5/2015,785.373319,0,0,0,6.92,51.077
...,...,...,...,...,...,...,...
2218,1/27/2021,NaN,0,0,0,3.33,39.664
2219,1/28/2021,NaN,0,0,0,1.67,195.314
2220,1/29/2021,NaN,0,0,0,-2.78,235.894
2221,1/30/2021,NaN,0,0,0,1.11,152.752


In [91]:
forecast = result.forecast.df[['Date', 'forecast']]
forecast_silverkite = forecast.iloc[-len(future_df):,:]

forecast_silverkite

,Date,forecast
2192,2021-01-01,750.528138
2193,2021-01-02,833.859730
2194,2021-01-03,752.893998
2195,2021-01-04,897.355507
2196,2021-01-05,778.988897
2197,2021-01-06,913.949976
2198,2021-01-07,825.744425
2199,2021-01-08,754.156193
2200,2021-01-09,821.351750
2201,2021-01-10,736.587352
